In [1]:
import tensorflow as tf
import numpy as np

In [2]:
sample = ' if you want you build a ship'
sample_set = list(set(sample))
char2idx = {c : i for i, c in enumerate(sample_set)}
idx2char = {i : c for i, c in enumerate(sample_set)}
print(char2idx,idx2char)

{'h': 0, 'd': 1, 'i': 2, 'y': 3, 'l': 4, 'f': 5, 'o': 6, 'b': 7, ' ': 8, 'p': 9, 'u': 10, 'n': 11, 's': 12, 'a': 13, 't': 14, 'w': 15} {0: 'h', 1: 'd', 2: 'i', 3: 'y', 4: 'l', 5: 'f', 6: 'o', 7: 'b', 8: ' ', 9: 'p', 10: 'u', 11: 'n', 12: 's', 13: 'a', 14: 't', 15: 'w'}


In [3]:
# hyper parameters
dic_size = len(char2idx)
rnn_hidden_size = len(char2idx)
num_classes = len(char2idx)
batch_size = 1
sequence_length = len(sample) -1
dic_size, rnn_hidden_size, num_classes, batch_size, sequence_length

(16, 16, 16, 1, 28)

In [4]:
sample_index = [char2idx[c] for c in sample]
x_data = [sample_index[:-1]]
y_data = [sample_index[1:]]

In [27]:
prediction_sample = 'if  build you want ship you a'
prediction_sample_index = [char2idx[c] for c in prediction_sample]
prediction_data = [prediction_sample_index[:-1]]

In [28]:
x_data[0][:5], y_data[0][:5], prediction_data[0][:5]

([8, 2, 5, 8, 3], [2, 5, 8, 3, 6], [2, 5, 8, 8, 7])

In [29]:
X = tf.placeholder(tf.int32, shape=[None, sequence_length])
Y = tf.placeholder(tf.int32, shape=[None, sequence_length])

In [30]:
X_one_hot = tf.one_hot(X, len(sample_set))
print(X_one_hot)

Tensor("one_hot_2:0", shape=(?, 28, 16), dtype=float32)


In [31]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
with tf.variable_scope("lstm_kernel", reuse=tf.AUTO_REUSE):
    outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)

In [32]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)

In [33]:
prediction = tf.argmax(outputs, axis=2)

In [34]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        l, _ = sess.run([loss, train], feed_dict={X:x_data, Y:y_data})
        if i % 100 == 0:
            result = sess.run(prediction, feed_dict={X:prediction_data})
            result_str=[idx2char[c] for c in np.squeeze(result)]
            print(i, "loss:", l, "prediction:", ''.join(result_str))
     

0 loss: 2.776608 prediction: inppaaaataataaaattattnna  aa
100 loss: 2.6274922 prediction:                             
200 loss: 2.5524058 prediction:                             
300 loss: 2.465875 prediction:                             
400 loss: 2.370521 prediction:                             
500 loss: 2.2953103 prediction:                        i   i
600 loss: 2.2231362 prediction:                             
700 loss: 2.1474636 prediction:                             
800 loss: 2.0513408 prediction: f                           
900 loss: 1.9903233 prediction: f                           
1000 loss: 1.9331586 prediction: f                           
1100 loss: 1.8880283 prediction: f                 u         
1200 loss: 1.8500211 prediction: f y               uu        
1300 loss: 1.8194603 prediction: f y               uuu       
1400 loss: 1.793905 prediction: f yy              uuuu i    
1500 loss: 1.7716563 prediction: f yy              uuuu i    
1600 loss: 1.7521662 pre

In [ ]:
prediction_sample = 'if  build you want ship you a'
